In [147]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime
from difflib import SequenceMatcher
import pymysql
from sqlalchemy import create_engine
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

In [109]:
def db_connection(host_name='ds'):
    host_url = "db.ds.mycelebs.com"
    user_nm = "j_eungg"
    passwd = "mycelebs"
    port_num = 3306
    db_name = "star_ko"
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8',
                           db=db_name, cursorclass=pymysql.cursors.DictCursor)
    # cursorclass=pymysql.cursors.DictCursor 추가 -> 데이터프레임으로 다루기 쉽게 딕셔너리 형태로 결과 반환해줌, cursor는 튜플형태
    # db=db,
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    return conn, engine

In [149]:
conn, engine = db_connection()

In [150]:
qry = 'SELECT * FROM star_ko_profile_url WHERE naver_url is NULL'
data = pd.read_sql(qry, conn)

In [151]:
data.head()

,skpu_pk,series_id,name,daum_url,naver_url,regist_date,update_date
0,12,12,GI,None,None,2021-03-26 05:00:00,NaT
1,14,14,GOT7,None,None,2021-03-26 05:00:00,NaT
2,29,29,강레오,None,None,2021-03-26 05:00:00,NaT
3,77,77,국카스텐,None,None,2021-03-26 05:00:00,NaT
4,82,82,글램,None,None,2021-03-26 05:00:00,NaT


In [152]:
len(data)

6303

In [153]:
data_list = data['series_id'].to_list()

In [157]:
number = int(input("네이버 url 수집할 개체 수 입력 :"))
ex = data_list[0:number]

mapping = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
naver_drv = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

for i in tqdm(ex) :
    conn, engine = db_connection()
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    qry = f'SELECT series_id, cd_name, cd_real_name, cd_gender, cd_category, cd_solr_search, cd_height, cd_weight, cd_birth, cd_agency, cd_belong, cd_team, cd_group, cd_debut, cd_academic, cd_career, cd_award FROM star_ko_data WHERE series_id={i} and cd_is_use=1'
    celebData = pd.read_sql(qry, con=conn)
    celebData = celebData.fillna('')
    
    try :
        series_id = celebData['series_id'][0]
        cd_name = celebData['cd_name'][0]
        cd_gender = celebData['cd_gender'][0]
        cd_category = celebData['cd_category'][0]
        cd_birth = celebData['cd_birth'][0]
        if cd_birth == '0000-00-00 00:00:00':
            cd_birth_ = ''
        elif cd_birth == '':
            cd_birth_ = ''
        else:
            cd_birth_ = cd_birth.strftime('%Y-%m-%d')
        cd_agency = celebData['cd_agency'][0]
        cd_belong = celebData['cd_belong'][0]
        cd_team = celebData['cd_team'][0]
        cd_group = celebData['cd_group'][0]
        cd_debut = celebData['cd_debut'][0]
        cd_debut = re.sub("\!|\'|\?", "", str(cd_debut))  # 특수문자 제거
        cd_height = celebData['cd_height'][0]
        cd_weight = celebData['cd_weight'][0]
        cd_career = celebData['cd_career'][0]
        cd_academic = celebData['cd_academic'][0]
        cd_award = celebData['cd_award'][0]
        
        display(celebData)
       
        
        naver_drv.get('https://people.search.naver.com/')
        naver_drv.find_element_by_id('nx_query').send_keys(cd_name)
        naver_drv.find_element_by_class_name('btn_window').click()
        
        naver_source = naver_drv.page_source
        naver_soup = BeautifulSoup(naver_source, "html.parser")
        
        result_footer = naver_soup.select('div[class=result_footer]')
        pages = result_footer[0].text
        pages = re.sub(r'[^0-9]', '', pages)
        
        
        for page in range(len(pages)) :
            
            people_url = []
            people = naver_soup.find_all('div', {'class': 'who'})

            # 한 페이지 내 인물 리스트 수집
            for p in range(len(people)):
                naver_base = 'https://people.search.naver.com/search.naver'
                people_url_ = naver_base + people[p].select('a')[0].get('href')
                people_url.append(people_url_)

            # 크롤링
            for pu in people_url :
                birth = year = month = day = agency = belong_not_celeb = belong_sport_team = belong_group = height = weight = debut = career = academic = awards = ''

                naver_html = requests.get(pu).text
                naver_soup2 = BeautifulSoup(naver_html, 'html.parser')

                profile_dsc = naver_soup2.select('div[class=profile_dsc]')           
                dt_in_profile_dsc_dsc = profile_dsc[0].select('dl[class=dsc]')[0].select('dt')
                dd_in_profile_dsc_dsc = profile_dsc[0].select('dl[class=dsc]')[0].select('dd')

                for row_len in range(len(dt_in_profile_dsc_dsc)):

                    title = dt_in_profile_dsc_dsc[row_len].text
                    inner = dd_in_profile_dsc_dsc[row_len]

                    if '출생' in title:
                        if ',' in inner.text:
                            tmp_birth, tmp_born_place = inner.text.split(',')[0].split(), inner.text.split(',')[1]
                        else:
                            tmp_birth = inner.text.split()
                        for t in tmp_birth:
                            if '년' in t:
                                year = re.sub("\D", "", t)
                            elif '월' in t:
                                month = re.sub("\D", "", t)
                            elif '일' in t:
                                day = re.sub("\D", "", t)
                        if year != '' and month != '' and day != '':
                            birth = "{0}-{1}-{2}".format(year.zfill(4), month.zfill(2), day.zfill(2))

                        if birth != '' and '00' in birth[5:]:
                            birth = ''

                    elif title == '소속사':
                        agency = inner.text.strip()

                    elif title == '소속' :
                        belong_not_celeb = inner.text

                    elif title == '소속팀' :
                        belong_sport_team = inner.text

                    elif '그룹' in title:
                        belong_group = inner.text

                    elif '신체' in title:
                        tmp_body = inner.text.split(',')
                        for it_tmp_body in tmp_body:
                            if 'cm' in it_tmp_body:
                                if height == '':
                                    height = re.sub("\D", "", it_tmp_body)
                            elif 'kg' in it_tmp_body:
                                if weight == '':
                                    weight = re.sub("\D", "", it_tmp_body)

                    elif '데뷔' in title:
                        debut = inner.text.strip()

                h3_in_profile_dsc_dsc = naver_soup2.select('div[class=record_wrap]')[0].select('h3[class=blind]')
                d1_in_profile_dsc_dsc = naver_soup2.select('div[class=record_wrap]')[0].select('dl')

                try :
                    for v in range(len(d1_in_profile_dsc_dsc)):
                        title = h3_in_profile_dsc_dsc[v].text
                        inner = d1_in_profile_dsc_dsc[v].select('p')

                        if '경력' in title:
                            career = inner[0].text

                        elif '학력' in title:
                            academic = inner[0].text

                        elif '수상' in title:
                            awards = inner[0].text
                except :
                    pass

                print("\n")
                print("---------- 네이버 데이터 ----------")
                print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
                      "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                
                # 생일 -> 데이터가 없으면
                if cd_birth_ == '' or birth == '' :

                    # 소속사 -> 데이터 없거나 다르면
                    if cd_agency == '' or agency == '' or cd_agency != agency :

                        # 소속 -> 데이터 없거나 다르면
                        if cd_belong == '' or belong_not_celeb == '' or cd_belong != belong_not_celeb :

                            # 소속팀 -> 데이터 없거나 다르면
                            if cd_team == '' or belong_sport_team == '' or cd_team != belong_sport_team :

                                # 그룹 -> 데이터 없거나 다르면
                                if cd_group == '' or belong_group == '' or cd_group != belong_group :

                                    # 데뷔 -> 데이터 없으면
                                    ratio = SequenceMatcher(None, debut, cd_debut).ratio()
                                    if cd_debut == '' or debut == '':

                                        # 경력사항 -> 데이터 없으면
                                        if cd_career == '' or career == '':

                                            # 학력 -> 데이터 없으면
                                            if cd_academic == '' or academic == '':

                                                # 수상내역 -> 데이터 없으면
                                                if cd_award == '' or awards == '':

                                                    # 키 & 몸무게 -> 데이터 없으면
                                                    if cd_height == '' and cd_weight == '':
                                                        pass
                                                    elif height == '' and weight == '':
                                                        pass
                                                    elif height == '' or cd_height == '' or weight == '' or cd_weight == '':
                                                        pass
                                                    # 키 & 몸무게 -> 데이터 있으면
                                                    elif height != '' and cd_height != '' and weight != '' and cd_weight != '':
                                                        if abs(float(height) - float(cd_height)) <= 2 and abs(float(weight) - float(cd_weight)) <= 2:
                                                            
#                                                             print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                                                   "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                                
                                                            print("[INFO] 키 & 몸무게 -> 매핑 성공!")
                                                            mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                                            mapping.append(mapping_data)
                                                            
                                                            update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                                            cursor.execute(update_qry)
                                                            conn.commit()
                                                            
                                                            raise NotImplementedError
                                                            
                                                        else :
                                                            pass

                                                # 수상내역 -> 데이터 있으면
                                                else :
                                                    for s in cd_award.split('|'):
                                                        awards_ratio = SequenceMatcher(None, s, awards).ratio()
                                                        # 수상내역 -> 데이터 70% 이상 비슷하면
                                                        if awards_ratio >= 0.7:
                                                            
#                                                             print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                                                   "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                                
                                                            print("[INFO] 수상내역 -> 매핑 성공!")
                                                            mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                                            mapping.append(mapping_data)
                                                            
                                                            update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                                            cursor.execute(update_qry)
                                                            conn.commit()
                                                            
                                                            raise NotImplementedError
                                                        # 수상내역 -> 데이터 70% 이상 비슷하지 않으면 패스    
                                                        else :
                                                            pass

                                            # 학력 -> 데이터 있으면
                                            else :
                                                for s in cd_academic.split('|'):
                                                    academic_ratio = SequenceMatcher(None, s, academic).ratio()
                                                    # 학력 -> 데이터 70% 이상 비슷하면
                                                    if academic_ratio >= 0.7:
                                                        
#                                                         print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                                                 "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)

                                                        print("[INFO] 학력 -> 매핑 성공!")
                                                        mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                                        mapping.append(mapping_data)
                                                        
                                                        update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                                        cursor.execute(update_qry)
                                                        conn.commit()
                                                        
                                                        raise NotImplementedError
                                                    # 학력 -> 데이터 70% 이상 비슷하지 않으면 패스    
                                                    else:
                                                        pass

                                        # 경력사항 -> 데이터 있으면
                                        else :
                                            for s in cd_career.split('|'):
                                                career_ratio = SequenceMatcher(None, s, career).ratio()
                                                # 경력사항 -> 데이터 70% 이상 비슷하면
                                                if career_ratio >= 0.7:
                                                    
#                                                     print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb, "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                                
                                                    print("[INFO] 경력사항 -> 매핑 성공!")
                                                    mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                                    mapping.append(mapping_data)
                                                    
                                                    update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                                    cursor.execute(update_qry)
                                                    conn.commit()
                                                    
                                                    raise NotImplementedError
                                                # 경력사항 -> 데이터 70% 이상 비슷하지 않으면 패스    
                                                else :
                                                    pass

                                    # 데뷔 -> 80% 이상 비슷하면 매핑
                                    elif ratio >= 0.8:
                                        
#                                         print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                               "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                                
                                        print("[INFO] 데뷔 -> 매핑 성공!")
                                        mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                        mapping.append(mapping_data)
                                        
                                        update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                        cursor.execute(update_qry)
                                        conn.commit()
                                        
                                        raise NotImplementedError

                                    # 데뷔 -> 80% 이상 비슷하지 않으면 패스
                                    elif ratio < 0.8:
                                        pass

                                # 그룹 -> 같으면 매핑
                                elif cd_group == belong_group :
                                    
#                                     print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                             "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                                
                                    print("[INFO] 그룹 -> 매핑 성공!")
                                    mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                    mapping.append(mapping_data)
                                    
                                    update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                    cursor.execute(update_qry)
                                    conn.commit()
                                    
                                    raise NotImplementedError

                            # 소속팀 -> 같으면 매핑    
                            elif cd_team == belong_sport_team :
                                
#                                 print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                     "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)

                                print("[INFO] 소속팀 -> 매핑 성공!")
                                mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                                mapping.append(mapping_data)
                                
                                update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                                cursor.execute(update_qry)
                                conn.commit()
                                
                                raise NotImplementedError

                        # 소속 -> 같으면 매핑
                        elif cd_belong == belong_not_celeb :
                            
#                             print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                     "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)

                            print("[INFO] 소속 -> 매핑 성공!")
                            mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                            mapping.append(mapping_data)
                            
                            update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                            cursor.execute(update_qry)
                            conn.commit()
                            
                            raise NotImplementedError

                    # 소속사 -> 같으면 매핑
                    elif cd_agency == agency :
                        
#                         print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                                 "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)
                                
                        print("[INFO] 소속사 -> 매핑 성공!")
                        mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                        mapping.append(mapping_data)
                        
                        update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                        cursor.execute(update_qry)
                        conn.commit()
                        
                        raise NotImplementedError

               # 생일 -> 같으면 매핑
                elif cd_birth_ == birth :
                    
#                     print("이름: ", cd_name, "| 생일: ", birth, "| 소속사: ", agency, "| 소속: ", belong_not_celeb,
#                         "| 소속팀: ", belong_sport_team, "| 그룹: ", belong_group, "| 데뷔:", debut, "| 키: ", height, "| 몸무게:", weight)          
                    
                    print("[INFO] 생일 -> 매핑 성공!")
                    mapping_data = {'series_id' : series_id, 'naver_url' : pu}
                    mapping.append(mapping_data)
                    
                    update_qry = f'UPDATE star_ko_profile_url SET naver_url=\'{pu}\' WHERE series_id={series_id};'
                    cursor.execute(update_qry)
                    conn.commit()
                    
                    raise NotImplementedError

                # 생일 -> 다르면 패스    
                elif cd_birth_ != birth :
                    pass
                
#             print("[INFO]", cd_name, "-", page, "페이지 크롤링 완료")
            try :
                naver_drv.find_element_by_class_name('next').click() # 다음 페이지
            except :
                pass # 다음 페이지 없음
            
    except Exception as e :
        print(e)
    
conn.close()
naver_drv.close()

네이버 url 수집할 개체 수 입력 :5




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
  0%|          | 0/5 [00:00<?, ?it/s]

,series_id,cd_name,cd_real_name,cd_gender,cd_category,cd_solr_search,cd_height,cd_weight,cd_birth,cd_agency,cd_belong,cd_team,cd_group,cd_debut,cd_academic,cd_career,cd_award
0,12,GI,,여자,가수,"(content_all:(""가수 지아이""~10 OR ""가수 GI""~10))",,,,,,,,,,,




---------- 네이버 데이터 ----------
이름:  GI | 생일:   | 소속사:  심통엔터테인먼트 | 소속:   | 소속팀:   | 그룹:   | 데뷔: 2013년 디지털 싱글 앨범 [Beatles] | 키:   | 몸무게: 


---------- 네이버 데이터 ----------
이름:  GI | 생일:  1993-06-25 | 소속사:  케이제이뮤직엔터테인먼트 | 소속:   | 소속팀:   | 그룹:   타겟  | 데뷔: 2018년 타겟 미니 앨범 [Alive] | 키:   | 몸무게: 


 20%|██        | 1/5 [00:01<00:07,  1.90s/it]

...



---------- 네이버 데이터 ----------
이름:  국카스텐 | 생일:   | 소속사:  인터파크 엔터테인먼트 | 소속:   | 소속팀:   | 그룹:   | 데뷔: 2008년 싱글 앨범 [Guckkasten] | 키:   | 몸무게: 
[INFO] 데뷔 -> 매핑 성공!



,series_id,cd_name,cd_real_name,cd_gender,cd_category,cd_solr_search,cd_height,cd_weight,cd_birth,cd_agency,cd_belong,cd_team,cd_group,cd_debut,cd_academic,cd_career,cd_award
0,82,글램,,여자,가수,"(content_all:((title:""글램""-title:""글램핑"")AND""기자""A...",,,,쏘스뮤직,,,,2012년 싱글 앨범 'Girls Be Ambitious',,,




---------- 네이버 데이터 ----------
이름:  글램 | 생일:   | 소속사:   | 소속:   | 소속팀:   | 그룹:   | 데뷔: 2012년 싱글 앨범 [Party (XXO)] | 키:   | 몸무게: 


---------- 네이버 데이터 ----------
이름:  글램 | 생일:   | 소속사:   | 소속:   | 소속팀:   | 그룹:   | 데뷔: 2004년 1집 앨범 [The Glam] | 키:   | 몸무게: 


100%|██████████| 5/5 [00:04<00:00,  1.03it/s]



---------- 네이버 데이터 ----------
이름:  글램 | 생일:   | 소속사:   | 소속:   | 소속팀:   | 그룹:   | 데뷔:  | 키:   | 몸무게: 


In [136]:
mapping_series_id = []

for m in mapping :
    mapping_ = m['series_id']
    mapping_series_id.append(mapping_)
    

In [137]:
non_mapping = []

for e in ex :
    if e not in mapping_series_id :
        non_mapping.append(e)

In [138]:
non_mapping

[12, 14]

In [ ]:
##### 매핑 안된 애들 url 대신 x 넣는거 짜야댐